In [1]:
import pgbquery as bq
import numpy as np
import pandas as pd
import os
import datetime
import operator
from multicorn import Qual
from sklearn.feature_extraction import DictVectorizer as DV

initiate the bcolz table

In [14]:
test = bq.ctable.create_ctable(test, rootdir='data/test')

TypeError: unbound method create_ctable() must be called with ctable instance as first argument (got ctable instance instead)

append some data

In [3]:
test = bz.ctable([bz.carray(['a', 'b', 'c']), 
                  bz.carray([1,2,3]), 
                  bz.carray([datetime.date(2016,1,2), datetime.date(2016,1,3), datetime.date(2016,1,4)])
                 ], ['letter', 'number', 'date'])

write data from memory to disk

In [6]:
test.addcol(['letter a', 'letter b', 'letter c'], 'desc')
test.addcol([True, True, False], name='booleans')

ValueError: 'desc' column already exists

load table to memory from disk

In [2]:
test = bq.open('/home/ralbright/data/test', mode='a')

In [3]:
dir(test)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_arr1',
 '_cparams',
 '_get_stats',
 '_iter',
 '_where',
 'addcol',
 'aggregate_groups',
 'append',
 'attrs',
 'auto_flush',
 'cache_factor',
 'cache_valid',
 'cbytes',
 'cols',
 'copy',
 'cparams',
 'create_agg_ctable',
 'create_ctable',
 'delcol',
 'dtype',
 'eval',
 'factorize_groupby_cols',
 'flush',
 'free_cachemem',
 'fromdataframe',
 'fromhdf5',
 'groupby',
 'is_in_ordered_subgroups',
 'iter',
 'len',
 'make_group_index',
 'mkdir_rootdir',
 'mode',
 'names',
 'nbytes',
 'ndim',
 'open_ctable',
 'resize',
 'rootdir',
 'shape',
 'size',
 'todataframe',
 'tohdf5',
 'trim',
 'unique',
 'where',
 'where_terms',
 'whereblocks']

In [6]:
def execute(quals, columns):
    filters = []
    for qual in quals:
        
       if not qual.is_list_operator:
            if type(qual.value) == str:
                value = "'%s'" % qual.value
            else:
                value = qual.value
            filters.append((str(qual.field_name), str(qual.operator), str(value)))
       else:
            op = qual.operator[0]
            optype = qual.operator[1]
            if op in ('=', '==') and optype == ANY:
                operator = 'IN'
            if op in ('!=', '<>') and optype == ANY:
                operator = 'NOT IN'
                filters.append((qual.fieldname, operator, qual.value))
                
    boolarr = test.where_terms(filters)
    for line in test.where(boolarr, outcols=test.names):
        yield line

In [7]:
line = execute([Qual('letter', '=', 'a')], test.names)

In [8]:
line.next()

row(letter='a', number=1, date=datetime.date(2016, 1, 2), desc='letter a', floats=56.43, booleans=True)

In [66]:
cols=
cols[0]= bz.zeros(0, 'S25')
cols[1] = bz.zeros(0, 'f8')


SyntaxError: invalid syntax (<ipython-input-66-1d26738a792b>, line 1)

In [43]:
mask = test.where_terms([('letter', '==', "'a'")])

In [44]:
w = test.where(mask)

In [45]:
w.next()

row(letter='a', number=1, date=datetime.date(2016, 1, 2), desc='letter a', floats=56.43, booleans=True)

In [17]:
import operator
operator.eq(test['letter'], 'a')

False

In [8]:
dir(bq.ctable)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_stats',
 '_iter',
 '_where',
 'addcol',
 'aggregate_groups',
 'append',
 'cache_factor',
 'cache_valid',
 'cbytes',
 'copy',
 'cparams',
 'create_agg_ctable',
 'create_ctable',
 'delcol',
 'dtype',
 'eval',
 'factorize_groupby_cols',
 'flush',
 'free_cachemem',
 'fromdataframe',
 'fromhdf5',
 'groupby',
 'is_in_ordered_subgroups',
 'iter',
 'make_group_index',
 'mkdir_rootdir',
 'names',
 'nbytes',
 'ndim',
 'open_ctable',
 'resize',
 'shape',
 'size',
 'todataframe',
 'tohdf5',
 'trim',
 'unique',
 'where',
 'where_pgterms',
 'where_terms',
 'whereblocks']

In [1]:
load_ext Cython

In [2]:
%%cython -a
import re2
import operator
dir(operator)
print operator.eq(1,2)

ImportError: libre2.so.0: cannot open shared object file: No such file or directory